# Capabilities in Dana

This tutorial covers the capabilities system in Dana, which provides a way to define and manage agent capabilities within the planning and reasoning layers.

## Learning Objectives

By the end of this tutorial, you will understand:

1. How to define agent capabilities
2. How to create custom capabilities
3. How to compose capabilities
4. How to manage capability discovery and registration
5. How to version and update capabilities
6. How to test and validate capabilities

## Prerequisites

- Basic understanding of Dana's 2-layer architecture (planning and reasoning)
- Familiarity with Python classes and inheritance
- Understanding of basic resource management concepts

## 1. Understanding Capabilities

Capabilities in Dana are modular components that define what an agent can do. They can be:

1. **Basic Capabilities**: Simple, single-function capabilities
2. **Composite Capabilities**: Combinations of multiple capabilities
3. **Domain-Specific Capabilities**: Specialized for particular domains

Capabilities are used by both the planning layer (to determine what can be done) and the reasoning layer (to implement how it's done).

Let's start by creating some basic capabilities:

In [ ]:
from dana.agent.capability import BaseCapability
from typing import Dict, Any, Optional

class DataAnalysisCapability(BaseCapability):
    """Example capability for data analysis."""
    def __init__(self, name: str = "data_analysis"):
        super().__init__(name)
        self.description = "Analyzes data using various statistical methods"
        self.version = "1.0.0"
        
    async def execute(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """Execute data analysis."""
        # Example implementation
        return {
            "mean": sum(data.get("values", [])) / len(data.get("values", [])),
            "count": len(data.get("values", []))
        }

class VisualizationCapability(BaseCapability):
    """Example capability for data visualization."""
    def __init__(self, name: str = "visualization"):
        super().__init__(name)
        self.description = "Creates visualizations from data"
        self.version = "1.0.0"
        
    async def execute(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """Create visualization."""
        # Example implementation
        return {
            "plot_type": data.get("plot_type", "line"),
            "data_points": len(data.get("values", []))
        }

## 2. Creating Custom Capabilities

Let's create a more complex capability that combines data analysis and visualization:

In [ ]:
class DataAnalysisAndVisualizationCapability(BaseCapability):
    """Composite capability combining data analysis and visualization."""
    def __init__(self, name: str = "data_analysis_and_visualization"):
        super().__init__(name)
        self.description = "Analyzes data and creates visualizations"
        self.version = "1.0.0"
        self.analysis_capability = DataAnalysisCapability()
        self.visualization_capability = VisualizationCapability()
        
    async def execute(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """Execute data analysis and create visualization."""
        # First analyze the data
        analysis_result = await self.analysis_capability.execute(data)
        
        # Then create visualization
        visualization_data = {
            "values": data.get("values", []),
            "plot_type": data.get("plot_type", "line"),
            "analysis": analysis_result
        }
        visualization_result = await self.visualization_capability.execute(visualization_data)
        
        return {
            "analysis": analysis_result,
            "visualization": visualization_result
        }

## 3. Capability Discovery and Registration

Capabilities need to be registered with the execution context to be used by the planning and reasoning layers. Let's see how this works:

In [ ]:
from dana.execution import ExecutionContext

# Create capabilities
analysis_capability = DataAnalysisCapability()
visualization_capability = VisualizationCapability()
composite_capability = DataAnalysisAndVisualizationCapability()

# Create execution context
context = ExecutionContext()

# Register capabilities
context.register_capability(analysis_capability)
context.register_capability(visualization_capability)
context.register_capability(composite_capability)

# List registered capabilities
print("Registered Capabilities:")
for capability in context.get_capabilities():
    print(f"- {capability.name} (v{capability.version}): {capability.description}")

## 4. Using Capabilities in Planning and Reasoning

Let's see how capabilities are used in both the planning and reasoning layers:

In [ ]:
from dana.execution.planning import Plan
from dana.execution.planning.plan_step import PlanStep
from dana.execution.reasoning import ChainOfThoughtStrategy

# Create a plan that uses our capabilities
plan = Plan(
    name="data_analysis_plan",
    description="Plan for analyzing and visualizing data",
    objective="Analyze and visualize manufacturing data"
)

# Create plan steps that use our capabilities
plan.add_step(PlanStep(
    name="analyze_data",
    description="Analyze manufacturing data",
    capability="data_analysis",
    expected_duration="1 day"
))

plan.add_step(PlanStep(
    name="visualize_results",
    description="Create visualizations of analysis results",
    capability="visualization",
    expected_duration="1 day"
))

# Create a reasoning strategy
reasoning_strategy = ChainOfThoughtStrategy()

# Execute the plan
result = context.execute_plan(plan, reasoning_strategy)

print("Plan Execution Result:")
print(f"- Status: {result.status}")
print(f"- Completed Steps: {result.completed_steps}")
print(f"- Total Duration: {result.total_duration}")

## 5. Versioning and Updates

Capabilities can be versioned and updated. Let's see how this works:

In [ ]:
class UpdatedDataAnalysisCapability(DataAnalysisCapability):
    """Updated version of the data analysis capability."""
    def __init__(self, name: str = "data_analysis"):
        super().__init__(name)
        self.version = "2.0.0"
        self.description = "Enhanced data analysis with additional statistical methods"
        
    async def execute(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """Execute enhanced data analysis."""
        # Enhanced implementation
        values = data.get("values", [])
        if not values:
            return {"error": "No data provided"}
            
        mean = sum(values) / len(values)
        variance = sum((x - mean) ** 2 for x in values) / len(values)
        std_dev = variance ** 0.5
        
        return {
            "mean": mean,
            "variance": variance,
            "std_dev": std_dev,
            "count": len(values)
        }

# Create and register the updated capability
updated_capability = UpdatedDataAnalysisCapability()
context.register_capability(updated_capability)

# Check version compatibility
print("Capability Versions:")
for capability in context.get_capabilities():
    if capability.name == "data_analysis":
        print(f"- {capability.name}: v{capability.version}")

## 6. Testing and Validation

It's important to test and validate capabilities. Let's see how to do this:

In [ ]:
import pytest

# Test data
TEST_DATA = {
    "values": [1, 2, 3, 4, 5],
    "plot_type": "line"
}

# Test the data analysis capability
async def test_data_analysis_capability():
    capability = DataAnalysisCapability()
    result = await capability.execute(TEST_DATA)
    
    assert "mean" in result
    assert "count" in result
    assert result["count"] == 5
    assert result["mean"] == 3.0

# Test the visualization capability
async def test_visualization_capability():
    capability = VisualizationCapability()
    result = await capability.execute(TEST_DATA)
    
    assert "plot_type" in result
    assert "data_points" in result
    assert result["plot_type"] == "line"
    assert result["data_points"] == 5

# Test the composite capability
async def test_composite_capability():
    capability = DataAnalysisAndVisualizationCapability()
    result = await capability.execute(TEST_DATA)
    
    assert "analysis" in result
    assert "visualization" in result
    assert result["analysis"]["mean"] == 3.0
    assert result["visualization"]["plot_type"] == "line"

# Run the tests
if __name__ == "__main__":
    pytest.main([__file__])

## Next Steps

In this tutorial, we've covered:

1. Understanding capabilities in Dana
2. Creating custom capabilities
3. Composing capabilities
4. Managing capability discovery and registration
5. Versioning and updating capabilities
6. Testing and validating capabilities

In the next tutorial, we'll explore the resources system in Dana, which provides access to various tools and services that capabilities can use.